# OpenAI Evals

[Evals](https://github.com/openai/evals) provide a framework for evaluating large language models (LLMs) or systems built using LLMs. It offers an existing registry of evals to test different dimensions of OpenAI models and the ability to write your own custom evals for use cases you care about. You can also use your data to build private evals which represent the common LLMs patterns in your workflow without exposing any of that data publicly.

If you are building with LLMs, creating high quality evals is one of the most impactful things you can do. Without evals, it can be very difficult and time intensive to understand how different model versions might effect your use case. 

## Easy Use of evals
After installing this tool, we can simply run evaluation in the command line by defining completion function (the use of models; prompting strategies…) and evaluation task (evaluation metrics, datasets and general protocols).

* `completion_fn`: I only want to evaluate openai Chat LLMs. Therefore, I can use any openai model id, e.g., “gpt-3.5-turbo”, “gpt-4”, “gpt-4–32k”. Here, I use gpt-3.5-turbo . But the evals framework provides general protocols for other LLM piplines and names them as completion functions, e.g., LangChain LLMs.
* `eval_task`: It refers to a YAML file in the evals.registry.evals directory. The file defines parameters for a specific evaluation task, e.g., evaluation data, evaluation metrics and prompting strategies. You can refer to Section 1: Specification File for details. Here, match_mmlu_machine_learning refers to a specification file discussed in the next bullet point.
```console
oaieval gpt-3.5-turbo test-match
```


## Scenario 1: Evalution of retriaval with different options

Let's assume that we have a chat completion that uses a CSV file for retriaval. This file should have only two columns: text and embedding. For each user query, it will find top {k} embeddings from CSV, which are the closest to the user's query, then add the corresponding 'text' of these embeddings to the system message to enrich the context. And the completion will reply accordingly. This is already implemented in [evals.completion_fns.retrieval:RetrievalCompletionFn](https://github.com/openai/evals/blob/main/evals/completion_fns/retrieval.py) class in evals.

We will register our own completion function using this class. It is also possible to implement a custom completion function class that inherits from events.api:CompletionFn. In this example, we will just use the existing 'RetrievalCompletionFn'. If you want to read more about Completion Functions, you can read [this document](https://github.com/openai/evals/blob/main/docs/completion-fns.md)

### Step 1: Setup retrieval data
While we are using RetrievalCompletionFn, we will use [president_birthdays.csv](./data/president_birthdays.csv). 
1. We will generate the 'text' column using the data in the file
2. We will will get the 'embeddings' using the text column
3. We will generate 'output/presidents_embeddings.csv' file only with 'text' and 'embedding' columns. This file will be the input for RetrievalCompletionFn.

In [13]:
import pandas as pd

input_datapath = "data/president_birthdays.csv"

df = pd.read_csv(input_datapath).rename(columns={" \"Name\"": "Name", " \"Month\"": "Month", " \"Day\"": "Day", " \"Year\"": "Year"}).set_index("Index")
df["text"] = df.apply(lambda r: f"{r['Name']} was born on {r['Month']}/{r['Day']}/{r['Year']}", axis=1)
display(df.head())

,Name,Day,Month,Year,text
Index,,,,,
1,"""George Washington""",22,2,1732.0,"""George Washington"" was born on 2/22/1732.0"
2,"""John Adams""",30,10,1735.0,"""John Adams"" was born on 10/30/1735.0"
3,"""Thomas Jefferson""",13,4,1743.0,"""Thomas Jefferson"" was born on 4/13/1743.0"
4,"""James Madison""",16,3,1751.0,"""James Madison"" was born on 3/16/1751.0"
5,"""James Monroe""",28,4,1758.0,"""James Monroe"" was born on 4/28/1758.0"


In [15]:
from openai import OpenAI
client = OpenAI()

def embed(text):
    return client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        ).data[0].embedding

df["embedding"] = df['text'].apply(embed)
df[["text", "embedding"]].to_csv("output/presidents_embeddings.csv")

,Name,Day,Month,Year,text,embedding
Index,,,,,,
1,"""George Washington""",22,2,1732.0,"""George Washington"" was born on 2/22/1732.0","[-0.0066544716246426105, 0.0005056292284280062..."
2,"""John Adams""",30,10,1735.0,"""John Adams"" was born on 10/30/1735.0","[-0.0015555905411019921, 0.040288060903549194,..."


### Step 2: Build your completion function

To register our completion function, we need to write a YAML file. You can check [presidents.yaml](./evals_registry/completion_fns/presidents.yaml). 

Let's explain the file content. RetrievalCompletionFn class takes 3 arguments:
1. **completion_fn**: we can simply pass a model name, e.g. 'gpt-4-turbo-preview' or pass another completions function. It is kind of chain, a completion function having another one as an input. 
2. **embeddings_and_text_path**: CSV file with text and embedding columns. It will be used for retrieval. 
3. **k**: Top k closest embeddings will be passed to the prompt. In our case it is 2, because the user will always ask questions similar to "Was Andrew Jackson born before William Harrison?", so we always need only 2 president.

Here we defined 3 different completion functions:
1. **cot/gpt-4-turbo-preview**: We will not use this one directly, but use it as an input to the 3rd one. It is based on [ChainOfThoughtCompletionFn](https://github.com/openai/evals/blob/main/evals/completion_fns/cot.py) class, adds chain of thought logic on top of gpt-4-turbo-preview.
1. **retrieval/presidents/gpt-4-turbo-preview**: for using 'gpt-4-turbo-preview' with retrieval.
2. **retrieval/presidents/cot/gpt-4-turbo-preview**: for using 'gpt-4-turbo-preview' with chain of thought logic, with retrieval.


In [18]:
# Open the file in read mode ('r')
with open('evals_registry/completion_fns/presidents.yaml', 'r') as file:
    # Read the file's content
    file_content = file.read()
    # Print the content
    print(file_content)

cot/gpt-4-turbo-preview:
  class: evals.completion_fns.cot:ChainOfThoughtCompletionFn
  args:
    cot_completion_fn: gpt-4-turbo-preview

retrieval/presidents/gpt-4-turbo-preview:
  class: evals.completion_fns.retrieval:RetrievalCompletionFn
  args:
    completion_fn: gpt-4-turbo-preview
    embeddings_and_text_path: ../../output/presidents_embeddings.csv
    k: 2

retrieval/presidents/cot/gpt-4-turbo-preview:
  class: evals.completion_fns.retrieval:RetrievalCompletionFn
  args:
    completion_fn: cot/gpt-4-turbo-preview
    embeddings_and_text_path: ../../output/presidents_embeddings.csv


### Step 3: Build your eval

An eval is simply a dataset and a choice of eval class. You have 3 options to build an eval:
1. Using one of the basic eval classes. Most common ones are 'Match', 'Includes' and 'FuzyMatch'. For a full list, you can check [here](https://github.com/openai/evals/tree/main/evals/elsuite/basic).
2. Using the [model graded eval class](https://github.com/openai/evals/blob/main/docs/eval-templates.md#the-model-graded-eval-template). Model graded eval means using an LLM model to evaluate the outputs of another LLM model. 
3. Creating your custom eval class. Most of the cases, this will not be necessary. So it is not covered under this notebook. But you can check [here](https://github.com/openai/evals/blob/main/docs/custom-eval.md), if you want to learn more.

Register the eval by adding a file to /evals/<eval_name>.yaml under registry folder (in our case it is 'evals_registry') using the elsuite registry format. For example, for a Match eval, it would be:

```console
<eval_name>:
  id: <eval_name>.dev.v0
  description: <description>
  metrics: [accuracy]

<eval_name>.dev.v0:
  class: evals.elsuite.basic.match:Match
  args:
    samples_jsonl: <eval_name>/samples.jsonl
```

Upon running the eval, the data will be searched for in data folder under registry. For example, if older/samples.jsonl is the provided filepath, the data is expected to be in evals_registry/data/older/samples.jsonl.

The naming convention for evals is in the form <eval_name>.<split>.<version>.

* <eval_name> is the eval name, used to group evals whose scores are comparable.
* <split> is the data split, used to further group evals that are under the same <base_eval>. E.g., "val", "test", or "dev" for testing.
* <version> is the version of the eval, which can be any descriptive text you'd like to use (though it's best if it does not contain .).
In general, running the same eval name against the same model should always give similar results so that others can reproduce it. Therefore, when you change your eval, you should bump the version.

In our sample scerio, we will go ahead with the first option and use 'Match'. You can check [older.yaml](./evals_registry/evals/older.yaml). 

In [19]:
# Open the file in read mode ('r')
with open('evals_registry/evals/older.yaml', 'r') as file:
    # Read the file's content
    file_content = file.read()
    # Print the content
    print(file_content)

older:
  id: older.dev.v0
  description: Test the model's ability to determine who is older.
  metrics: [accuracy]
older.dev.v0:
  class: evals.elsuite.basic.match:Match
  args:
    samples_jsonl: older/older.jsonl


In [17]:
import os

# Replace with path to your registry
registry_path = "evals/registry/evals"
os.makedirs(registry_path, exist_ok=True)
with open(registry_path + "/older.yaml", "w") as f:
    f.write(registry_yaml)

In [ ]:
# GPT-3.5-turbo base: accuracy 0.7
!oaieval gpt-4-turbo-preview born-first --max_samples 10 --registry_path ./evals/registry

In [ ]:
# GPT-3.5-turbo with retrieval: accuracy 0.9 -> The failure mode here is the retrieved president is incorrect: Andrew Johnson vs Andrew Jackson
!oaieval retrieval/presidents/gpt-4-turbo-preview born-first --max_samples 10 --registry_path ./evals/registry

In [ ]:
# GPT-3.5-turbo with retrieval and chain-of-thought: accuracy 1.0
!oaieval retrieval/presidents/cot/gpt-4-turbo-preview born-first --max_samples 10 --registry_path ./evals/registry